Data Preperation

In [28]:
import pandas as pd
import numpy as np
from itertools import chain

In [29]:
training = pd.read_csv('HMAHCC_COMP.csv')

C:\Users\cgn31\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
class DataPrep:
    def __init__(self, df):
        self.raw_df = df
    
    def seperate_frame(self):
        new_diagnosis_col = ['New diagnosis - CPD',
                         'New diagnosis - Hypertension', 
                         'New diagnosis - Top 5', 
                         'New diagnosis - CAD', 
                         'New diagnosis - CHF', 
                         'New diagnosis - Diabetes']

        call_col = ['Inbound Call by Prov',
                    'Inbound Call by Mbr',
                    'Inbound Call by Other']

        non_rx_claim_col = ['Surgery',
                            'Fully Paid Claim']

        rx_claim_col = ['RX Claim - New Drug',
                        'RX Claim - First Time Mail Order']
        
        self.new_diagnosis = self.raw_df[self.raw_df['event_descr'].isin(new_diagnosis_col)]

        self.new_diagnosis = self.new_diagnosis.drop(['event_attr6',
                                                      'event_attr7',
                                                      'event_attr8',
                                                      'event_attr9',
                                                      'event_attr10',
                                                      'PAY_DAY_SUPPLY_CNT',
                                                      'PAYABLE_QTY',
                                                      'MME',
                                                      'DRUG_TYPE',
                                                      'Specialty',
                                                      'Specialty2',
                                                      'Specialty3'],
                                                       axis = 1)

        self.new_diagnosis.columns = ['id',
                                 'event_descr',
                                 'diagnosis',
                                 'place_of_treatment',
                                 'charge_amount',
                                 'net_paid_amount',
                                 'member_responsible_amount',
                                 'Days']
        
        self.call = self.raw_df[self.raw_df['event_descr'].isin(call_col)]

        self.call = self.call.drop(['event_attr6',
                          'event_attr7',
                          'event_attr8',
                          'event_attr9',
                          'event_attr10',
                          'PAY_DAY_SUPPLY_CNT',
                          'PAYABLE_QTY',
                          'MME',
                          'DRUG_TYPE',
                          'Specialty',
                          'Specialty2',
                          'Specialty3'],
                           axis = 1)

        self.call.columns = ['id',
                        'event_descr',
                        'call_category',
                        'inquiry_reason_description',
                        'disposition_description',
                        'origin',
                        'location',
                        'Days']
        
        self.non_rx_claim = self.raw_df[self.raw_df['event_descr'].isin(non_rx_claim_col)]

        self.non_rx_claim = self.non_rx_claim.drop(['event_attr6',
                                          'event_attr7',
                                          'event_attr8',
                                          'event_attr9',
                                          'event_attr10',
                                          'PAY_DAY_SUPPLY_CNT',
                                          'PAYABLE_QTY',
                                          'MME',
                                          'DRUG_TYPE',
                                          'Specialty',
                                          'Specialty2',
                                          'Specialty3'],
                                           axis = 1)

        self.non_rx_claim.columns = ['id',
                                'event_descr',
                                'diagnosis',
                                'place_of_treatment',
                                'charge_amount',
                                'net_paid_amount',
                                'member_responsible_amount',
                                'Days']
        
        self.new_provider = self.raw_df[self.raw_df['event_descr']=='New provider']
        self.new_provider = self.new_provider.iloc[:,:2]
        
        self.rx_claim = self.raw_df[self.raw_df['event_descr'].isin(rx_claim_col)]

        self.rx_claim = self.rx_claim.drop(['event_attr6',
                          'event_attr7'],
                           axis = 1)

        self.rx_claim.columns = ['id',
                            'event_descr',
                            'gpi_drug_group6_id',
                            'gpi_drug_class_description',
                            'brand_name',
                            'drug_group_id',
                            'generic_name',
                            'drug_group_description',
                            'member_responsible_amount',
                            'gpi_drug_group8_id',
                            'Days',
                            'PAY_DAY_SUPPLY_CNT',
                            'PAYABLE_QTY',
                            'MME',
                            'DRUG_TYPE',
                            'Specialty',
                            'Specialty2',
                            'Specialty3']
        
        self.rx_paid = self.raw_df[self.raw_df['event_descr']=='RX Claim - Paid']

        self.rx_paid = self.rx_paid.drop(['event_attr2',
                                'event_attr7'],
                                 axis = 1)

        self.rx_paid.columns = ['id',
                           'event_descr',
                           'gpi_drug_class_description',
                           'rx_cost',
                           'net_paid_amount',
                           'brand_name',
                           'drug_group_description',
                           'generic_name',
                           'member_responsible_amount',
                           'gpi_drug_group8_id',
                           'Days',
                           'PAY_DAY_SUPPLY_CNT',
                           'PAYABLE_QTY',
                           'MME',
                           'DRUG_TYPE',
                           'Specialty',
                           'Specialty2',
                           'Specialty3']
        
        self.rx_reject = self.raw_df[self.raw_df['event_descr']=='RX Claim - Rejected']

        self.rx_reject = self.rx_reject.drop(['PAY_DAY_SUPPLY_CNT',
                                    'PAYABLE_QTY',
                                    'MME',
                                    'DRUG_TYPE',
                                    'Specialty',
                                    'Specialty2',
                                    'Specialty3'],
                                     axis =1)

        self.rx_reject.columns = ['id',
                             'event_descr',
                             'status_code',
                             'diagnosis',
                             'cob',
                             'claim_tier',
                             'brand_name',
                             'generic_name',
                             'ndc_id',
                             'pay_day_supply_count',
                             'member_responsible_amount',
                             'gpi_drug_group8_id',
                             'Days']
        
    def get_opioid_data(self):
        self.opioid_data = self.raw_df[self.raw_df['PAY_DAY_SUPPLY_CNT'].notnull()]
        self.opioid_data_grouped = self.opioid_data.groupby(by='id')
        
    def LTOT(self):
        
        def get_LTOT(ID):
            try:
                group = self.opioid_data_grouped.get_group(ID)
                frame = group[['Days', 'PAY_DAY_SUPPLY_CNT']].drop_duplicates()
                frame = frame[frame['Days']>=0]
                frame['drugs until'] = (frame['Days'] + frame['PAY_DAY_SUPPLY_CNT']).astype(int)
                frame['range'] = frame.apply(lambda x : range(x['Days'].astype(int),x['drugs until'].astype(int)),1)

                concat = concatenated = chain(*list(frame['range']))
                concat = set(concat)

                day_frame = pd.DataFrame(columns = ['Days', 'Has Drug?'])
                day_frame['Days'] = range(max(concat)+1)
                day_frame['Has Drug?'] = (day_frame['Days'].isin(concat))

                for n in range(180,len(day_frame)+1):
                    frame_slice = day_frame.iloc[n-180:n]
                    drug_days = np.sum(frame_slice['Has Drug?'])

                    if drug_days >= 162:
                        return (True, n-180, n)

                return (False, np.nan, np.nan)
            except:
                return (np.nan, np.nan, np.nan)
        
        id_list = self.raw_df['id'].drop_duplicates().values
        
        response_variable = pd.DataFrame(id_list, columns = ['id'])
        response_variable['LTOT'] = response_variable['id'].map(get_LTOT)
        
        response_variable[['LTOT', 'Begining Date', 'End Date']] = \
        pd.DataFrame(response_variable['LTOT'].tolist(), index = response_variable.index)
        
        self.response_variable = response_variable

In [71]:
training_class = DataPrep(training)

In [72]:
training_class.get_opioid_data()

In [73]:
training_class.LTOT()

In [74]:
training_class.response_variable

,id,LTOT,Begining Date,End Date
0,ID10010854159,False,NaN,NaN
1,ID10013863216,True,0.0,180.0
2,ID10024447278,True,0.0,180.0
3,ID1002482139,True,0.0,180.0
4,ID1003386406,False,NaN,NaN
5,ID10036289882,False,NaN,NaN
6,ID10036954381,False,NaN,NaN
7,ID10055739763,True,224.0,404.0
8,ID10074598346,False,NaN,NaN
9,ID10081072715,False,NaN,NaN


In [75]:
dir(training_class)

['LTOT',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'get_opioid_data',
 'opioid_data',
 'opioid_data_grouped',
 'raw_df',
 'response_variable',
 'seperate_frame']

From here we work with the data:

2